In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [3]:
upperframe=[]  
num_pages=1 # The total number of pages you can scrape from PolitiFact was 613 at the time of this project
for page in range(1,num_pages+1):
    
    print(f'Scraping Page: {page}')
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)

    response=requests.get(url)   
    print(response.status_code)
        
    if response.status_code != 200:
        print(f'Failed to scrape page {page}')
        
    soup=BeautifulSoup(response.text,'lxml')
    frame=[]
    table=soup.find_all('li',attrs={'class':'o-listicle__item'})
    
    for row in table:
        Statement = row.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Article = "https://www.politifact.com" + row.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = row.find('div',attrs={'class':'m-statement__body'}).find('footer').text.strip()
        Source = row.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = row.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Article,Date,Source,Label))
    upperframe.extend(frame)
    
    time.sleep(5)

data=pd.DataFrame(upperframe, columns=['Statement','Article','Date','Source','Label'])
data.head()

Scraping Page: 1
200


,Statement,Article,Date,Source,Label
0,"During the debate, Kamala Harris said, “that’s...",https://www.politifact.com/factchecks/2020/oct...,"By Ciara O'Rourke • October 12, 2020",Facebook posts,barely-true
1,Image shows Nancy Pelosi drunk and being kisse...,https://www.politifact.com/factchecks/2020/oct...,"By Rebecca Catalanello • October 12, 2020",Facebook posts,false
2,Says Dr. Anthony Fauci said of Trump’s pandemi...,https://www.politifact.com/factchecks/2020/oct...,"By Bill McCarthy • October 11, 2020",Donald Trump,barely-true
3,Says U.S. Rep. Lauren Underwood described riot...,https://www.politifact.com/factchecks/2020/oct...,"By Kiannah Sepeda-Miller • October 11, 2020",Jim Oberweis,false
4,"""If Biden becomes president and cannot complet...",https://www.politifact.com/factchecks/2020/oct...,"By Ilana Strauss • October 10, 2020",Facebook posts,false


In [32]:
data.shape

(17730, 5)

In [34]:
data.isnull().sum()

Statement    0
Article      0
Date         0
Source       0
Label        0
dtype: int64

In [40]:
data['Article'][17729]

'https://www.politifact.com/factchecks/2007/nov/26/barack-obama/no-mandate-makes-universal-claim-tough/'

In [47]:
data.to_csv('politifact_news.csv', index = False)

In [4]:
upperframe_2=[]
for i in range(0, len(data['Article'])):
    
    url = data['Article'][i]
    response=requests.get(url)   
    
    if i%200 == 0:
        print(f'Scraping Page: {i}')
        print(url)
        print(response.status_code)
        time.sleep(10)
    
    elif response.status_code != 200:
        print(f'Failed to scrape page {i}')
        upperframe_2.append(['NA'])
        continue
        
    soup=BeautifulSoup(response.text,'lxml')
    frame=[]
    table=soup.find('article',attrs={'class':'m-textblock'})
    for row in table.find_all('p')[:-3]:
        info = row.text.strip().split('\n')
        frame.append(info)
        result = []
        [result.extend(_) for _ in frame]
        result = [' '.join(result)]
    upperframe_2.append(result)
    
articles=pd.DataFrame(upperframe_2, columns = ['Story'])
articles.head()

In [169]:
upperframe_2[5] = ['NA']

In [5]:
articles=pd.DataFrame(upperframe_2, columns = ['Story'])
articles.head(10)

In [172]:
articles.shape

(17730, 1)

In [209]:
complete_data = pd.concat([articles,data], axis = 1)

In [211]:
new_date = []
for i in range(0, len(complete_data['Article'])):
    new_date.append(complete_data['Article'][i][38:49])

In [212]:
complete_data['date'] = new_date

In [6]:
complete_data.head()

In [214]:
complete_data.to_csv('complete_politifact_news.csv', index = False)